In [1]:
import casadi as cs
from urdf_parser_py.urdf import URDF, Pose
import os # For current directory
import urdf2casadi.urdf2casadi.urdfparser
import urdf2casadi.urdf2casadi.geometry.plucker as plucker
import numpy as np
import PyKDL as kdl
import kdl_parser.kdl_parser_py.kdl_parser_py.urdf as kdlurdf
from timeit import Timer, timeit, repeat
import rbdl

ok, ur_tree = kdlurdf.treeFromFile('./urdf2casadi/examples/urdf/test.urdf')
#ok, ur_tree = kdlurdf.treeFromFile('pantilt.urdf')
asd = urdf2casadi.urdf2casadi.urdfparser.URDFparser()
robot_desc = asd.from_file("./urdf2casadi/examples/urdf/test.urdf")
#robot_desc = asd.from_file("pantilt.urdf")
root = 'base_link'
tip = 'wrist_3_link'
ur_chain = ur_tree.getChain(root,tip)
rbdlmodel = rbdl.loadModel("ur5_rbdl.urdf")

The root link base_link has an inertia specified in the URDF, but KDL does not support a root link with an inertia.  As a workaround, you can add an extra dummy link to your URDF.
./urdf2casadi/examples/urdf/test.urdf


In [2]:
jlist, names, qmax, qmin = asd.get_joint_info(root, tip)
njoints = len(jlist)
print njoints
print len(qmax)

H_ur = kdl.JntSpaceInertiaMatrix(6)

grav = kdl.Vector()
grav[2] = -9.81

q = kdl.JntArray(njoints)
qdot = kdl.JntArray(njoints)

q_np = np.zeros(njoints)
qdot_np = np.zeros(njoints)
qddot_np = np.zeros(njoints)
tau_rbdl = np.zeros(njoints)

q_sx = cs.SX.zeros(njoints)
print q_sx
qdot_sx = cs.SX.zeros(njoints)

q_none = [None]*njoints
qdot_none = [None]*njoints
qddot_none = [None]*njoints


tau_kdl = kdl.JntArray(njoints)
grav_kdl = kdl.JntArray(njoints)

nitr = 50000
nrepeat = 3

M_rbdl = (rbdlmodel.q_size, rbdlmodel.q_size)
M_rbdl = np.zeros(M_rbdl)



6
6
@1=0, [@1, @1, @1, @1, @1, @1]


In [14]:
tau_u2c = asd.get_inverse_dynamics_RNEA(root, tip, grav)
#q_none = [1., 1., 1., 1., 1., 1.]
#qdot_none = [1., 1., 1., 1., 1., 1.]
#qddot_none = [1., 1., 1., 1., 1., 1.]
def idrnea_u2c():   
    for j in range(njoints):
        q_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qdot_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qddot_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2

    tau_u2c(q_none, qdot_none, qddot_none)

#timeit_fdcrba = timeit("fdcrba_u2c()", setup = "from __main__ import fdcrba_u2c", number = 50000)
#print timeit_fdcrba
#print timeit_fdcrba/50000
timeit_idrnea_repeat = repeat("idrnea_u2c()", setup = "from __main__ import idrnea_u2c", repeat = 8, number = 50000)
print timeit_idrnea_repeat

[2.473716974258423, 2.3878560066223145, 2.4360198974609375, 2.3810958862304688, 2.467115879058838, 2.3593969345092773, 2.4299259185791016, 2.4418880939483643]


In [17]:
def id_rbdl():   
    for j in range(njoints):
        q_np[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qdot_np[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qddot_np[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2

    rbdl.InverseDynamics(rbdlmodel, q_np, qdot_np, qddot_np, tau_rbdl)

timeit_idrbdl_repeat = repeat("id_rbdl()", setup = "from __main__ import id_rbdl", repeat = 8, number = 50000)
print timeit_idrbdl_repeat

[1.990778923034668, 2.0226800441741943, 2.913455009460449, 2.3995749950408936, 1.9109230041503906, 2.2883129119873047, 2.560305118560791, 2.7022480964660645]
